<a href="https://colab.research.google.com/github/yellowssnake/221_zlotin/blob/main/assets/Notebooks/TinyStories_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers datasets accelerate nvidia-ml-py3
!pip install bitsandbytes

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.9 MB/s eta 0:00:00


In [2]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


# Description

In this assignment, you will train a language model (LM) using the TinyStories dataset, focusing on optimizing model performance within the constraints of Google Colab’s hardware. For the sake of speed, we will do it on the part of the dataset.

```
Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun.
Beep was a healthy car because he always had good fuel....
```

Your objective is to maximize the size of the model without exceeding the available computational resources (~ 16GB VRAM). You could start with the Hugging Face Transformers library and experiment with various memory optimization techniques, such as (but not limited to):

 * Different batch size
 * Different optimizer
 * Gradient accumulation
 * Activation checkpointing
 * CPU offloading
 * 8bit optimizers

You have a baseline of training gpt-2 model prepared in this  colab notebook. You can easily switch it to opt-350m, opt-1.3b, gpt2 etc. You can find a great beginner-level guide on the topic [here](https://huggingface.co/docs/transformers/v4.18.0/en/performance).

```
A long time ago in a galaxy far far away... a little girl named Lily was playing in the garden. She was so excited! She wanted to explore the garden and see what was around her.
Suddenly, she heard a loud noise. Lily looked up and saw a big, hairy creature. Lily was so excited! She ran to the creature and grabbed it by the arm. The creature was so big and hairy that Lily couldn't help but laugh.
```

![](https://hse24.fmin.xyz/gpt2_generation.jpeg)

You have to fill this table with your description/observations.

| Setup | # of parameters | GPU peak memory, MB | Final eval loss | Batch Size | Time to run 5 epochs, s | Generation example | Comment |
|:---:|:---:|:---:|:---:|:---:|:---:|:---------:|:---------:|
| Baseline (OPT-125M) | 125 M | 9044 | 1.928 | 8 | 442.34 | `A long time ago in a galaxy far far away... there was a little girl named Lily. She was three years old and loved to explore. One day, she decided to go for a walk in the park. Lily was so excited to go for a walk. She asked her mom, "What do you want to do?" Her mom smiled and said, "I want to explore the galaxy." Lily was so excited to explore the galaxy.` |  |
| Baseline (GPT2-S) | 124 M | 13016 | 2.001 | 8 | 487.75 | `A long time ago in a galaxy far far away... a little girl named Lily was playing in the garden. She was so excited! She wanted to explore the garden and see what was around her. Suddenly, she heard a loud noise. Lily looked up and saw a big, hairy creature. Lily was so excited! She ran to the creature and grabbed it by the arm. The creature was so big and hairy that Lily couldn't help but laugh.` | The generation seems more interesting, despite the fact, that eval loss is higher. |
| opt-1.3B | 1.3B | 13400 | 1.97(переобучилась, оптимум 1.58) |  1(эффективный - 8)| 2100 | A long time ago in a galaxy far far away... there was a little girl named Lily. She was three years old and loved to explore. One day, she decided to go on an adventure and explore the galaxy.

Lily looked around and noticed something strange. She noticed that some of the stars were not normal stars, but were instead strange stars with strange shapes and colours.

Lily was curious and wanted to know what these strange stars were. So, she decided to |  |
| звучит лучше чем прошлая, на одной эпохе было бы еще лучше |  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |  |
|  |  |  |  |  |  |  |  |

For each unique trick for memory optimization, you will get 4 points (maximum 20 points). A combination of tricks is not counted as a unique trick, but will, probably, be necessary to train big models. The maximum grade is bounded with the size of the trained model:

* If the model size you train is <= 125M - you can get a maximum of 8 points.
* If the model size you train is 126M <= 350M - you can get a maximum of 12 points.
* If the model size you train is 350M <= 1B - you can get a maximum of 16 points.
* If you fit 1B model or more - you can get a maximum 20 points.

# Baseline

In [1]:
import torch

torch.cuda.synchronize()
torch.cuda.empty_cache()  # Clears the cache
torch.cuda.reset_peak_memory_stats()  #

from transformers import AutoModelForCausalLM, AutoTokenizer, \
    TrainingArguments, Trainer, logging, DataCollatorForLanguageModeling
from datasets import load_dataset
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

print("💎 Before training:")
print_gpu_utilization()

# Suppress less critical logs
logging.set_verbosity_error()

# Load the dataset with both training and evaluation splits
train_dataset = load_dataset("roneneldan/TinyStories", split="train[:500]")
eval_dataset = load_dataset("roneneldan/TinyStories", split="train[500:1000]")

HF_cardname = "openai-community/gpt2"
# HF_cardname = "openai-community/gpt2-medium"
# HF_cardname = "openai-community/gpt2-large"
# HF_cardname = "openai-community/gpt2-XL"
# HF_cardname = "facebook/opt-125m"
# HF_cardname = "facebook/opt-350m"
HF_cardname = "facebook/opt-1.3b"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(HF_cardname, use_fast=False)
EOS_TOKEN = tokenizer.eos_token

# Ensure the tokenizer has a padding token, set EOS_TOKEN as padding token if not present
if tokenizer.pad_token is None:
    tokenizer.pad_token = EOS_TOKEN

# Function to process the dataset
def formatting_func(examples):
    inputs = [tokenizer(text + EOS_TOKEN, truncation=True, max_length=512, padding="max_length", return_tensors="pt") for text in examples['text']]
    return {'input_ids': [input['input_ids'].squeeze() for input in inputs], 'labels': [input['input_ids'].squeeze() for input in inputs]}

# Process the datasets
processed_train_dataset = train_dataset.map(formatting_func, batched=True, remove_columns=["text"])
processed_eval_dataset = eval_dataset.map(formatting_func, batched=True, remove_columns=["text"])

# Initialize Data Collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

print("💎 Dataset loaded")
print_gpu_utilization()

# Define and load the model
model = AutoModelForCausalLM.from_pretrained(HF_cardname)

print("💎 Model loaded")
print_gpu_utilization()

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=1,
    num_train_epochs=5,
    weight_decay=0.01,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    fp16 =True,
    optim="adafactor",
)

# Initialize the trainer with the data collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_train_dataset,
    eval_dataset=processed_eval_dataset,
    data_collator=data_collator
)

# Start training
trainer.train()

def print_summary(trainer):
    # Access training result metrics directly from the trainer state
    print(f"💎 Training time: {trainer.state.log_history[-1]['train_runtime']:.2f} seconds")
    print(f"Samples/second: {trainer.state.log_history[-1]['train_samples_per_second']:.2f}")
    print_gpu_utilization()
    num_params = sum(p.numel() for p in trainer.model.parameters() if p.requires_grad)
    print(f"Total Trainable Parameters: {num_params}")

print_summary(trainer)

💎 Before training:
GPU memory occupied: 367 MB.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

💎 Dataset loaded
GPU memory occupied: 367 MB.
💎 Model loaded
GPU memory occupied: 367 MB.


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: grigory-zlotin (grigory-zlotin-hse-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'eval_loss': 1.5854524374008179, 'eval_runtime': 35.716, 'eval_samples_per_second': 13.999, 'eval_steps_per_second': 1.764, 'epoch': 1.0}
{'eval_loss': 1.5866097211837769, 'eval_runtime': 35.1151, 'eval_samples_per_second': 14.239, 'eval_steps_per_second': 1.794, 'epoch': 2.0}
{'eval_loss': 1.7188352346420288, 'eval_runtime': 35.1866, 'eval_samples_per_second': 14.21, 'eval_steps_per_second': 1.79, 'epoch': 3.0}
{'eval_loss': 1.854783296585083, 'eval_runtime': 35.2911, 'eval_samples_per_second': 14.168, 'eval_steps_per_second': 1.785, 'epoch': 4.0}
{'eval_loss': 1.9774389266967773, 'eval_runtime': 35.6913, 'eval_samples_per_second': 14.009, 'eval_steps_per_second': 1.765, 'epoch': 5.0}
{'train_runtime': 2064.3869, 'train_samples_per_second': 1.211, 'train_steps_per_second': 0.153, 'train_loss': 0.9427784753224207, 'epoch': 5.0}
💎 Training time: 2064.39 seconds
Samples/second: 1.21
GPU memory occupied: 13961 MB.
Total Trainable Parameters: 1315758080


In [2]:
# Encode the prompt text
input_ids = tokenizer.encode(
    "A long time ago in a galaxy far far away...",
    return_tensors="pt").cuda()

# Generate text using the model
output_ids = model.generate(input_ids, max_length=100)

# Decode the generated ids to text
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print the generated text
print(generated_text)

A long time ago in a galaxy far far away... there was a little girl named Lily. She was three years old and loved to explore. One day, she decided to go on an adventure and explore the galaxy.

Lily looked around and noticed something strange. She noticed that some of the stars were not normal stars, but were instead strange stars with strange shapes and colours.

Lily was curious and wanted to know what these strange stars were. So, she decided to
